In [1]:
import pandas as pd
#import geopandas as gpd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.formula.api as smf
from scipy.stats import pearsonr
import warnings
warnings.simplefilter(action='ignore')

In [2]:
data_dir = "../intermediate/"
figures_dir = "../figures/"
output_dir = "../output/"
data1_dir = "../data/"
df = pd.read_csv(f"{data1_dir}cbsa2fipsxw.csv")

In [3]:
df

,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,33860,NaN,388.0,"Montgomery, AL",Metropolitan Statistical Area,NaN,"Montgomery-Selma, AL",Autauga County,Alabama,1,1,Central
1,19300,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1,3,Central
2,21640,NaN,NaN,"Eufaula, AL-GA",Micropolitan Statistical Area,NaN,NaN,Barbour County,Alabama,1,5,Central
3,13820,NaN,142.0,"Birmingham, AL",Metropolitan Statistical Area,NaN,"Birmingham-Cullman-Talladega, AL",Bibb County,Alabama,1,7,Outlying
4,13820,NaN,142.0,"Birmingham, AL",Metropolitan Statistical Area,NaN,"Birmingham-Cullman-Talladega, AL",Blount County,Alabama,1,9,Outlying
...,...,...,...,...,...,...,...,...,...,...,...,...
1910,41980,NaN,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,NaN,"San Juan-Bayamón, PR",Vega Alta Municipio,Puerto Rico,72,143,Central
1911,41980,NaN,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,NaN,"San Juan-Bayamón, PR",Vega Baja Municipio,Puerto Rico,72,145,Central
1912,38660,NaN,434.0,"Ponce, PR",Metropolitan Statistical Area,NaN,"Ponce-Coamo, PR",Villalba Municipio,Puerto Rico,72,149,Outlying
1913,41980,NaN,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,NaN,"San Juan-Bayamón, PR",Yabucoa Municipio,Puerto Rico,72,151,Central


In [4]:
gdp = pd.read_csv(f"{data1_dir}CAGDP1__ALL_AREAS_2001_2023.csv", encoding='latin-1')

In [5]:
last_4_indices = gdp.index[-4:]  
gdp = gdp.drop(last_4_indices)

In [6]:
gdp

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"""00000""",United States,,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,14230726000,14472712000,...,18261714000,18799622000,19141672000,19612102000,20193896000,20715671000,20267585000,21494798000,22034828000,22671096000
1,"""00000""",United States,,CAGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,72.561,73.795,...,93.115,95.857,97.601,100,102.967,105.627,103.342,109.6,112.353,115.597
2,"""00000""",United States,,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,10581929000,10929108000,...,17608138000,18295019000,18804913000,19612102000,20656516000,21539982000,21354105000,23681171000,26006893000,27720709000
3,"""01000""",Alabama,5,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,168448296,172430379,...,206070026,208950294,212862789,216615470,220808767,225272775,222288753,233726620,238556458,245354674
4,"""01000""",Alabama,5,CAGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,77.764,79.602,...,95.132,96.461,98.268,100,101.936,103.997,102.619,107.899,110.129,113.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9529,"""97000""",Rocky Mountain,7,CAGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,66.507,67.417,...,90.651,94.157,96.237,100,104.559,109.558,109.182,116.884,120.681,125.65
9530,"""97000""",Rocky Mountain,7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,338172137,347931550,...,602131039,621890048,641047020,681310123,730567674,776330214,780157535,880955513,986528188,1057159025
9531,"""98000""",Far West,8,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,2463451014,2514956943,...,3360531450,3516768389,3629300282,3797440495,3956948041,4115306459,4055843532,4352482955,4411936147,4525827912
9532,"""98000""",Far West,8,CAGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,64.871,66.228,...,88.495,92.609,95.572,100,104.2,108.371,106.805,114.616,116.182,119.181


In [7]:
unique_descriptions = gdp['Description'].unique()
print(unique_descriptions)

['Real GDP (thousands of chained 2017 dollars) '
 'Chain-type quantity indexes for real GDP '
 'Current-dollar GDP (thousands of current dollars) ']


In [8]:
current_dollar_gdp = gdp[gdp['Description'] == 'Current-dollar GDP (thousands of current dollars) ']

In [9]:
current_dollar_gdp

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
2,"""00000""",United States,,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,10581929000,10929108000,...,17608138000,18295019000,18804913000,19612102000,20656516000,21539982000,21354105000,23681171000,26006893000,27720709000
5,"""01000""",Alabama,5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,123534517,128380896,...,197064403,203113340,208824280,216615470,226263784,234798213,235325321,260018429,284949136,304935567
8,"""01001""","Autauga, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,760973,800045,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
11,"""01003""","Baldwin, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,3128482,3369358,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
14,"""01005""","Barbour, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,642028,637765,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9521,"""94000""",Plains,4,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,673288427,699597124,...,1159084109,1196563655,1217270085,1250759343,1301099969,1340848043,1336823238,1476248606,1620951934,1726903182
9524,"""95000""",Southeast,5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,2276728106,2373091556,...,3726579792,3901056533,4037867547,4210089181,4411751306,4611953719,4609791490,5132905366,5685333127,6122626696
9527,"""96000""",Southwest,6,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,1106271358,1134836906,...,2168451055,2163072239,2169483346,2286160711,2464729990,2550062656,2484272963,2856515882,3295092219,3498332461
9530,"""97000""",Rocky Mountain,7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,338172137,347931550,...,602131039,621890048,641047020,681310123,730567674,776330214,780157535,880955513,986528188,1057159025


In [10]:
us = pd.read_csv(f"{data1_dir}CAGDP1_US_2001_2023.csv", encoding='latin-1')

In [11]:
us

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"""00000""",United States,,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,1.423073e+10,1.447271e+10,...,1.826171e+10,1.879962e+10,1.914167e+10,1.961210e+10,2.019390e+10,2.071567e+10,2.026758e+10,2.149480e+10,2.203483e+10,2.267110e+10
1,"""01000""",Alabama,5,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,1.684483e+08,1.724304e+08,...,2.060700e+08,2.089503e+08,2.128628e+08,2.166155e+08,2.208088e+08,2.252728e+08,2.222888e+08,2.337266e+08,2.385565e+08,2.453547e+08
2,"""02000""",Alaska,8,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,4.001438e+07,4.190467e+07,...,5.330325e+07,5.368115e+07,5.346386e+07,5.355092e+07,5.247958e+07,5.237748e+07,5.033277e+07,5.145415e+07,5.076711e+07,5.405968e+07
3,"""04000""",Arizona,6,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,2.308856e+08,2.382571e+08,...,3.017212e+08,3.085828e+08,3.190084e+08,3.330990e+08,3.463983e+08,3.595767e+08,3.650277e+08,3.950359e+08,4.102284e+08,4.223996e+08
4,"""05000""",Arkansas,5,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,9.552873e+07,9.855660e+07,...,1.214094e+08,1.215325e+08,1.230349e+08,1.238826e+08,1.263712e+08,1.272200e+08,1.283409e+08,1.374636e+08,1.395607e+08,1.428606e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"""98000""",Far West,8,CAGDP1,1.0,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,2.463451e+09,2.514957e+09,...,3.360531e+09,3.516768e+09,3.629300e+09,3.797440e+09,3.956948e+09,4.115306e+09,4.055844e+09,4.352483e+09,4.411936e+09,4.525828e+09
60,Note: See the included footnote file.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,CAGDP1: County and MSA gross domestic product ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,"Last updated: December 4, 2024-- new statistic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
us_fips_set = set(us["GeoFIPS"].unique())
current_dollar_gdp_filtered = current_dollar_gdp[~current_dollar_gdp["GeoFIPS"].isin(us_fips_set)]


In [13]:
df["fipsstatecode"] = df["fipsstatecode"].astype(str)
df["fipscountycode"] = df["fipscountycode"].astype(str)

df["fipsstatecode"] = df["fipsstatecode"].str.zfill(2) 
df["fipscountycode"] = df["fipscountycode"].str.zfill(3)

In [14]:
df["countyfips"] = df["fipsstatecode"] + df["fipscountycode"]

In [15]:
df

,cbsacode,metropolitandivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty,countyfips
0,33860,NaN,388.0,"Montgomery, AL",Metropolitan Statistical Area,NaN,"Montgomery-Selma, AL",Autauga County,Alabama,01,001,Central,01001
1,19300,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,01,003,Central,01003
2,21640,NaN,NaN,"Eufaula, AL-GA",Micropolitan Statistical Area,NaN,NaN,Barbour County,Alabama,01,005,Central,01005
3,13820,NaN,142.0,"Birmingham, AL",Metropolitan Statistical Area,NaN,"Birmingham-Cullman-Talladega, AL",Bibb County,Alabama,01,007,Outlying,01007
4,13820,NaN,142.0,"Birmingham, AL",Metropolitan Statistical Area,NaN,"Birmingham-Cullman-Talladega, AL",Blount County,Alabama,01,009,Outlying,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,41980,NaN,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,NaN,"San Juan-Bayamón, PR",Vega Alta Municipio,Puerto Rico,72,143,Central,72143
1911,41980,NaN,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,NaN,"San Juan-Bayamón, PR",Vega Baja Municipio,Puerto Rico,72,145,Central,72145
1912,38660,NaN,434.0,"Ponce, PR",Metropolitan Statistical Area,NaN,"Ponce-Coamo, PR",Villalba Municipio,Puerto Rico,72,149,Outlying,72149
1913,41980,NaN,490.0,"San Juan-Bayamón-Caguas, PR",Metropolitan Statistical Area,NaN,"San Juan-Bayamón, PR",Yabucoa Municipio,Puerto Rico,72,151,Central,72151


In [16]:

current_dollar_gdp_filtered["GeoFIPS"] = (
    current_dollar_gdp_filtered["GeoFIPS"].astype(str)
)
current_dollar_gdp_filtered["GeoFIPS"] = (
    current_dollar_gdp_filtered["GeoFIPS"]
    .str.replace('"', '', regex=False)   
    .str.strip()                  
)

current_dollar_gdp_filtered["GeoFIPS"] = (
    current_dollar_gdp_filtered["GeoFIPS"].str.zfill(5)
)

# 查看结果
current_dollar_gdp_filtered.head()


,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
8,01001,"Autauga, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,760973,800045,...,1569120,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642
11,01003,"Baldwin, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,3128482,3369358,...,6034727,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468
14,01005,"Barbour, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,642028,637765,...,779299,765000,757473,761761,789103,793769,787697,868828,931174,926685
17,01007,"Bibb, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,217109,224813,...,381354,377535,392287,406741,408791,465977,502089,530401,571231,610429
20,01009,"Blount, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,622473,637704,...,928552,987510,925988,987967,1064218,1055980,973485,1162103,1266269,1261107


In [17]:
urrent_dollar_gdp_filtered = current_dollar_gdp_filtered.merge(
    df[["countyfips", "cbsacode"]],  
    how="left",                      
    left_on="GeoFIPS",             
    right_on="countyfips"          
)

In [18]:
urrent_dollar_gdp_filtered.drop(columns="countyfips", inplace=True)

In [19]:
urrent_dollar_gdp_filtered

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,cbsacode
0,01001,"Autauga, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,760973,800045,...,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642,33860.0
1,01003,"Baldwin, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,3128482,3369358,...,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468,19300.0
2,01005,"Barbour, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,642028,637765,...,765000,757473,761761,789103,793769,787697,868828,931174,926685,21640.0
3,01007,"Bibb, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,217109,224813,...,377535,392287,406741,408791,465977,502089,530401,571231,610429,13820.0
4,01009,"Blount, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,622473,637704,...,987510,925988,987967,1064218,1055980,973485,1162103,1266269,1261107,13820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,56037,"Sweetwater, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,2314242,2027268,...,3674881,3500949,3769766,3822174,3796389,3403365,3845561,4311769,4498470,40540.0
3114,56039,"Teton, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,1139422,1205930,...,2212117,2341110,2527529,2618310,2761710,2851799,3906137,4225374,4642946,27220.0
3115,56041,"Uinta, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,643723,683950,...,926737,915204,951381,964606,924127,820273,921651,1062478,1113995,21740.0
3116,56043,"Washakie, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,251982,252290,...,407534,364765,342555,354292,360514,362761,389459,415865,439977,NaN


In [20]:
urrent_dollar_gdp_filtered["cbsacode"] = urrent_dollar_gdp_filtered["cbsacode"].astype(str)

In [21]:
urrent_dollar_gdp_filtered

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,cbsacode
0,01001,"Autauga, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,760973,800045,...,1729098,1806246,1762558,1826642,1814228,1841867,1954645,2318914,2452642,33860.0
1,01003,"Baldwin, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,3128482,3369358,...,6492574,6983037,7382558,7935575,8558537,8770138,9847424,11097699,12071468,19300.0
2,01005,"Barbour, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,642028,637765,...,765000,757473,761761,789103,793769,787697,868828,931174,926685,21640.0
3,01007,"Bibb, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,217109,224813,...,377535,392287,406741,408791,465977,502089,530401,571231,610429,13820.0
4,01009,"Blount, AL",5,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,622473,637704,...,987510,925988,987967,1064218,1055980,973485,1162103,1266269,1261107,13820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,56037,"Sweetwater, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,2314242,2027268,...,3674881,3500949,3769766,3822174,3796389,3403365,3845561,4311769,4498470,40540.0
3114,56039,"Teton, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,1139422,1205930,...,2212117,2341110,2527529,2618310,2761710,2851799,3906137,4225374,4642946,27220.0
3115,56041,"Uinta, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,643723,683950,...,926737,915204,951381,964606,924127,820273,921651,1062478,1113995,21740.0
3116,56043,"Washakie, WY",7,CAGDP1,3.0,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,251982,252290,...,407534,364765,342555,354292,360514,362761,389459,415865,439977,nan


In [22]:
year_columns = ["2011", "2012", "2013", "2014", "2015",
                "2016", "2017", "2018", "2019", "2020",
                "2021", "2022"]

df_long = urrent_dollar_gdp_filtered.melt(
    id_vars=["cbsacode"],       
    value_vars=year_columns,   
    var_name="year",            
    value_name="gdp"           
)

df_long = df_long[["year", "cbsacode", "gdp"]]

In [23]:
df_long = df_long.sort_values(by=["cbsacode", "year"])
df_long.head(20)

,year,cbsacode,gdp
2371,2011,10100.0,1897577
2387,2011,10100.0,240801
5489,2012,10100.0,1961624
5505,2012,10100.0,201869
8607,2013,10100.0,2191225
8623,2013,10100.0,275175
11725,2014,10100.0,2164838
11741,2014,10100.0,252105
14843,2015,10100.0,2203877
14859,2015,10100.0,213888


In [24]:
df_long["gdp"] = (
    df_long["gdp"]
    .str.replace(",", "", regex=False)  # 如果有千位分隔逗号，先删除
    .str.strip()                        # 去除前后空格
)

# 然后将其转为数值
df_long["gdp"] = pd.to_numeric(df_long["gdp"], errors="coerce")


In [25]:
df_cbsa_agg = (
    df_long
    .groupby(["cbsacode", "year"], as_index=False)  
    .agg({"gdp": "sum"})                       
)

print(df_cbsa_agg.head())

  cbsacode  year        gdp
0  10100.0  2011  2138378.0
1  10100.0  2012  2163493.0
2  10100.0  2013  2466400.0
3  10100.0  2014  2416943.0
4  10100.0  2015  2417765.0


In [26]:
factor = pd.read_csv(f"{data_dir}cleaned_dataset.csv")

In [27]:
factor

,Year,cbsacode,Sample_Size,Data_value,population,population.1,depressed,poor_or_fair_health,uninsured_adults,primary_care_physicians,...,population.2,population.3,rural,not_proficient_in_english,diabetes_prevalence,mental_health_providers,median_household_income,driving_alone_to_work,some_college,children_in_single-parent_households
0,2011,10740.0,2880,21.78,857903.0,857903.0,186851.2734,97397.000813,204089.060206,0.009477,...,857903.0,857903.0,87515.802000,73774.102000,38302.509293,175.779084,182856.0,668947.187000,391848.399342,76095.770766
1,2012,10740.0,2876,21.22,857903.0,857903.0,182047.0166,98867.369479,181737.900941,0.009477,...,857903.0,857903.0,87515.802000,73315.926435,40653.313909,175.779084,177323.0,673517.867082,396390.755662,78336.432256
2,2013,10740.0,1748,20.25,898642.0,898642.0,181975.0050,99962.935999,190855.622994,0.007162,...,898642.0,898642.0,80094.108418,38077.779861,42799.899999,491.829340,173348.0,707463.296363,424621.686590,82138.068083
3,2014,10740.0,1664,22.79,901700.0,901700.0,205497.4300,101291.149001,197894.709070,0.007741,...,901700.0,901700.0,80031.849930,37514.220729,43652.497000,1721.414683,176687.0,713888.065111,435973.474580,83334.047363
4,2015,10740.0,1300,20.84,902797.0,902797.0,188142.8948,101774.221004,183300.028562,0.007662,...,902797.0,902797.0,79873.777674,36982.845598,46869.105002,2951.000027,180701.0,719458.881619,439084.182453,84936.372654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,2018,49340.0,270,18.85,819589.0,819589.0,154492.5265,90496.890726,24980.988229,0.008436,...,819589.0,819589.0,150724.764567,25403.710589,63012.236002,3618.999973,69064.0,664973.771765,435945.923379,53259.036194
860,2019,49340.0,263,20.10,826116.0,826116.0,166049.3160,91544.178776,24675.663998,0.008265,...,826116.0,826116.0,151925.098561,26300.645008,62440.608004,3839.999975,70222.0,666465.128961,443790.757368,53987.244725
861,2020,49340.0,257,18.31,830839.0,830839.0,152126.6209,90776.424072,26506.035692,0.008217,...,830839.0,830839.0,152793.671789,27727.434829,59049.629999,4180.000029,71767.0,668849.331389,448932.182905,52873.043592
862,2021,49340.0,302,20.31,830622.0,830622.0,168699.3282,96836.196463,26124.153421,0.008368,...,830622.0,830622.0,152753.764868,27653.157830,60464.884001,4324.999996,77795.0,663029.330339,451952.707668,38850.507986


In [28]:
print(factor.columns)

Index(['Year', 'cbsacode', 'Sample_Size', 'Data_value', 'population',
       'population.1', 'depressed', 'poor_or_fair_health', 'uninsured_adults',
       'primary_care_physicians', 'adult_smoking', 'adult_obesity',
       'unemployment', 'low_birthweight', 'sexually_transmitted_infections',
       'excessive_drinking', 'population.2', 'population.3', 'rural',
       'not_proficient_in_english', 'diabetes_prevalence',
       'mental_health_providers', 'median_household_income',
       'driving_alone_to_work', 'some_college',
       'children_in_single-parent_households'],
      dtype='object')


In [29]:
factor["year"] = factor["Year"].astype(int)
df_cbsa_agg["year"] = df_cbsa_agg["year"].astype(int)

In [30]:
factor["cbsacode"] = factor["cbsacode"].astype(str)

df_cbsa_agg["cbsacode"] = df_cbsa_agg["cbsacode"].astype(str)

merged_df = factor.merge(df_cbsa_agg, how="left", on=["year", "cbsacode"])


In [31]:
merged_df

,Year,cbsacode,Sample_Size,Data_value,population,population.1,depressed,poor_or_fair_health,uninsured_adults,primary_care_physicians,...,rural,not_proficient_in_english,diabetes_prevalence,mental_health_providers,median_household_income,driving_alone_to_work,some_college,children_in_single-parent_households,year,gdp
0,2011,10740.0,2880,21.78,857903.0,857903.0,186851.2734,97397.000813,204089.060206,0.009477,...,87515.802000,73774.102000,38302.509293,175.779084,182856.0,668947.187000,391848.399342,76095.770766,2011,37141102.0
1,2012,10740.0,2876,21.22,857903.0,857903.0,182047.0166,98867.369479,181737.900941,0.009477,...,87515.802000,73315.926435,40653.313909,175.779084,177323.0,673517.867082,396390.755662,78336.432256,2012,37226565.0
2,2013,10740.0,1748,20.25,898642.0,898642.0,181975.0050,99962.935999,190855.622994,0.007162,...,80094.108418,38077.779861,42799.899999,491.829340,173348.0,707463.296363,424621.686590,82138.068083,2013,37168847.0
3,2014,10740.0,1664,22.79,901700.0,901700.0,205497.4300,101291.149001,197894.709070,0.007741,...,80031.849930,37514.220729,43652.497000,1721.414683,176687.0,713888.065111,435973.474580,83334.047363,2014,38596658.0
4,2015,10740.0,1300,20.84,902797.0,902797.0,188142.8948,101774.221004,183300.028562,0.007662,...,79873.777674,36982.845598,46869.105002,2951.000027,180701.0,719458.881619,439084.182453,84936.372654,2015,39874734.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,2018,49340.0,270,18.85,819589.0,819589.0,154492.5265,90496.890726,24980.988229,0.008436,...,150724.764567,25403.710589,63012.236002,3618.999973,69064.0,664973.771765,435945.923379,53259.036194,2018,44248636.0
860,2019,49340.0,263,20.10,826116.0,826116.0,166049.3160,91544.178776,24675.663998,0.008265,...,151925.098561,26300.645008,62440.608004,3839.999975,70222.0,666465.128961,443790.757368,53987.244725,2019,45806712.0
861,2020,49340.0,257,18.31,830839.0,830839.0,152126.6209,90776.424072,26506.035692,0.008217,...,152793.671789,27727.434829,59049.629999,4180.000029,71767.0,668849.331389,448932.182905,52873.043592,2020,46263981.0
862,2021,49340.0,302,20.31,830622.0,830622.0,168699.3282,96836.196463,26124.153421,0.008368,...,152753.764868,27653.157830,60464.884001,4324.999996,77795.0,663029.330339,451952.707668,38850.507986,2021,50885550.0


In [33]:
merged_df.to_csv(f'../output/analysis_dataset.csv',index=False)